In [1]:
# after installation and kernel upgrade restart needed
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import keras_nlp
import tensorflow as tf
import shutil
import keras

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")

# set training and testing data paths
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [15]:
# create datasets
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

train_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)

val_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

test_ds = tf.keras.utils.text_dataset_from_directory(
    test_dir, batch_size=batch_size)

class_labels = train_ds.class_names
print("\nClass names:", class_labels)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.

Class names: ['neg', 'pos']


## Custom vocabulary from the IMDB dataset

In [16]:
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    train_ds.map(lambda x, y: x),
    vocabulary_size=30_000,
    lowercase=True,
    strip_accents=True,
    reserved_tokens=["[PAD]", "[START]", "[END]", "[MASK]", "[UNK]"],
)
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=True,
    strip_accents=True,
    oov_token="[UNK]",
)

In [17]:
packer = keras_nlp.layers.StartEndPacker(
    start_value=tokenizer.token_to_id("[START]"),
    end_value=tokenizer.token_to_id("[END]"),
    pad_value=tokenizer.token_to_id("[PAD]"),
    sequence_length=512,
)

def preprocess(x, y):
    token_ids = packer(tokenizer(x))
    return token_ids, y

imdb_train_prepr = train_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

imdb_val_prepr = val_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

imdb_test_prepr = test_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

print(imdb_train_prepr.unbatch().take(1).get_single_element())

(<tf.Tensor: shape=(512,), dtype=int32, numpy=
array([    1,   353,   200,   182,    98, 12237, 10513,    11,    61,
         198,    16,    51,   143,   225,   105,   104,   100,   118,
         209,   112,    99,  1393,    16,    97,    96,   182,  1798,
          18,   102,    11,    61,    43,  2413,  2711,   928,   517,
       16557,   459,    98,  2175,    17,  2099,  1823,  1886,    16,
         210,    96,  1802,    98,   126,  2931, 12577,   459,    98,
        7559,    97, 13915,   395,   107,   289,    43,   267,   107,
         191,    98,    96,   195,    18,    96,   298,   100,   874,
          16,    96,   155,   100,  3227,    97,   531,    16,    97,
          96,   365,  1692,   100,   371,   295,   992,    18,  1138,
          16,  3603,  6067,   137,   115,   169,   183,  1571,   113,
         372,    97,  2037,   100,   462,    17,    99,    17,  1292,
          99,   170,   222,    55, 14882,   141,   113,    16,   150,
         138,   114,   843,    96,   428,  

## Design a tiny transformer

In [28]:
token_id_input = keras.Input(
    shape=(None,),
    dtype="int32",
    name="token_ids",
)

outputs = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=len(vocab),
    sequence_length=packer.sequence_length,
    embedding_dim=64,
)(token_id_input)

outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=2,
    intermediate_dim=128,
    dropout=0.1,
)(outputs)

# use "[START]" token to classify
outputs = keras.layers.Dense(2)(outputs[:, 0, :])
model = keras.Model(
    inputs=token_id_input,
    outputs=outputs,
)

model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ token_ids (InputLayer)          │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_2  │ (None, None, 64)       │     1,342,912 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_6           │ (None, None, 64)       │        33,472 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ get_item_7 (GetItem)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,376,514 (5.25 MB)

 Trainable params: 1,376,514 (5.25 MB)

 Non-trainable params: 0 (0.00 B)

## Train the transformer directly on the classification objective

In [29]:
class TriangularScheduler(keras.optimizers.schedules.LearningRateSchedule):
    """Linear ramp up for 'warmup' steps, then linear decay to 0 at 'total steps'."""
    def __init__(self, rate, warmup, total):
        self.rate = rate
        self.warmup = warmup
        self.total = total

    def get_config(self):
        config = {"rate": self.rate, "warmup": self.warmup, "total": self.total}
        return config

    def __call__(self, step):
        step = keras.ops.cast(step, dtype="float32")
        rate = keras.ops.cast(self.rate, dtype="float32")
        warmup = keras.ops.cast(self.warmup, dtype="float32")
        total = keras.ops.cast(self.total, dtype="float32")

        warmup_rate = rate * step / self.warmup
        cooldown_rate = rate * (total - step) / (total - warmup)
        triangular_rate = keras.ops.minimum(warmup_rate, cooldown_rate)
        return keras.ops.maximum(triangular_rate, 0.0)

epochs = 15
batch_size = 32
steps_per_epoch = train_ds.cardinality().numpy()
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(0.1 * num_train_steps)
initial_learning_rate = 5e-5

warmup_schedule = TriangularScheduler(initial_learning_rate, warmup_steps, num_train_steps)

In [30]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(warmup_schedule),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True,
)

e_stop = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
model.fit(
    imdb_train_prepr, validation_data=imdb_val_prepr, epochs=15,
    callbacks=[e_stop],
)

Epoch 1/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - loss: 0.7141 - sparse_categorical_accuracy: 0.4912 - val_loss: 0.6889 - val_sparse_categorical_accuracy: 0.5076
Epoch 2/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.6874 - sparse_categorical_accuracy: 0.5404 - val_loss: 0.5697 - val_sparse_categorical_accuracy: 0.7522
Epoch 3/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.5183 - sparse_categorical_accuracy: 0.7478 - val_loss: 0.4467 - val_sparse_categorical_accuracy: 0.7786
Epoch 4/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.3627 - sparse_categorical_accuracy: 0.8421 - val_loss: 0.3343 - val_sparse_categorical_accuracy: 0.8538
Epoch 5/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.2869 - sparse_categorical_accuracy: 0.8826 - val_loss: 0.3115 - val_sparse_categorical_accuracy: 0.8676
Epoch 6/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.2564 - sparse_categorical_accuracy: 0.8977 - val_loss: 0.2877 - val_sparse_categorical_accuracy:

In [31]:
model.evaluate(imdb_test_prepr)

782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.3019 - sparse_categorical_accuracy: 0.8815


[0.31072551012039185, 0.8772000074386597]